# be humble view bumble

In [2]:
import convwale

In [48]:
import importlib
importlib.reload(convwale)

<module 'convwale' from '/Users/Thomsn/Desktop/island_in_the_sun/python/2021_03_bumbleview/bumbleview/bumbleview/convwale.py'>

## Upload a floral emission spectrum

Please add a csv file (tab, comma or semicolon seperated) without header and rownames. The first column should reflect the wavelengths in nm, the following columns, the emmission spectra of each sample.

In [3]:
wl_uploader = convwale.input_flowers()

FileUpload(value={}, accept='.csv', description='Upload')

Please add an additional csv file with meta information about all the samples. Four columns of information are implemented: "Genus", "Species", "Leaf area", "Additional information/sample number". All those columns can be used for other kinds of information, but the terminology will stay like that until the end of the script.

In [4]:
meta_uploader = convwale.input_flowers()

FileUpload(value={}, accept='.csv', description='Upload')

In [49]:
wl_df = convwale.parse_flowers(wl_uploader)
meta_df = convwale.parse_flowers(meta_uploader)

flowers = convwale.new_floral_spectra(wl_df, meta_df)

## Select genus and leaf area

In [6]:
genus_choice = flowers.select_key()

Dropdown(description='Genus:', options=(('Rhododendron', 0), ('Primula', 1), ('Gentiana', 2), ('Silene_acaulis…

In [7]:
area_choice = flowers.select_key(key="area", genus_choice=genus_choice)

Dropdown(description='Area:', options=(('ventr', 0), ('dors', 1), (None, 2)), value=0)

In [12]:
genus = convwale.get_dropdown_value(genus_choice)
area = convwale.get_dropdown_value(area_choice)

## Plot the physical spectra

In [ ]:
fig = flowers.plot_wl_spectra(genus, area)
fig.show()

In [50]:
flowers.plot_all_inclusive(plot_type="wl_spectra")

/Users/Thomsn/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3641: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
/Users/Thomsn/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3641: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


## Plot insect vision

### Hexagon

In [ ]:
fig = flowers.plot_hexagon(genus=genus, area=area)
fig.show()

In [44]:
flowers.plot_all_inclusive(plot_type="hexagon")

### Triangle

In [ ]:
fig = flowers.plot_triangle(genus=genus, area=area)
fig.show()

In [45]:
flowers.plot_all_inclusive(plot_type="triangle")

## Download all plots

In [51]:
flowers.download_data()

/Users/Thomsn/Desktop/island_in_the_sun/python/2021_03_bumbleview/bumbleview/bumbleview/bumble_view_20210313210918156216.zip

## Clean up temporary directory

In [47]:
flowers.close_temporary_dir()